This notebook extracts ground points from classified las file, remove outliers and and claculate statistics of ground elevations, and optionally generate DTM rasters from the processed las files. The elevations extracted may be used as a gap filling ground elevation level for FFH estimation.

In [ ]:
import pdal
import json
import numpy as np
import os
import geopandas as gpd
import glob
from lidar.point_cloud_processings import process_extract_ground_elevations
import matplotlib.pyplot as plt

### Input and output

In [ ]:
building_points_file=r'/home/ubuntu/lavender_floor_height/output/Final_Wagga_training_samples_pano_metadata_clipping.geojson'
las_files_folder = r"/mnt/floorheightvolume/lidar_Wagga/clipped/"

out_folder=r"/mnt/floorheightvolume/lidar_Wagga/clipped_DTM/"
os.makedirs(out_folder, exist_ok=True)
out_building_points=r'/home/ubuntu/lavender_floor_height/output/Final_Wagga_training_samples_pano_metadata_clipping_elevations.geojson'

### Load building points

In [ ]:
gdf_building_points=gpd.read_file(building_points_file)
gdf_building_points=gdf_building_points[gdf_building_points["USAGE"]=="Residential"].reset_index(drop=True)

### Test processing with one example

In [ ]:
# gdf_building_points[gdf_building_points['UFI']==1624]

In [ ]:
i=1
building_ufi=gdf_building_points.iloc[i]['UFI']
las_file_path=glob.glob(las_files_folder+'*'+'_UFI_'+str(building_ufi)+'.las')[0]
las_file_path

In [ ]:
resolution=0.1 # resolution of output DTM raster
output_tiff=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_DTM_filtered.tif')) # output file name

In [ ]:
# Get CRS from source file first
pipeline_info = {
    "pipeline": [{
        "type": "readers.las",
        "filename": las_file_path
    }]
}
info_pipeline = pdal.Pipeline(json.dumps(pipeline_info))
info_pipeline.execute()
metadata = info_pipeline.metadata
crs = metadata['metadata']['readers.las']['srs']['horizontal']

In [ ]:
stats=process_extract_ground_elevations(las_file_path=las_file_path, resolution=resolution, crs=crs, output_tiff=output_tiff)

In [ ]:
# Print the statistics
print("Ground Elevation Statistics:")
for key, value in stats.items():
    print(f"{key:>12}: {value:.3f}")

### Record the stats for future use

In [ ]:
gdf_updated=gdf_building_points.copy()
for attr in list(stats.keys()):
    if attr not in gdf_updated.columns:
        gdf_updated[attr]=np.nan
for attr in list(stats.keys()):
    gdf_updated.at[i,attr]=stats[attr]

### Run for all buildings

In [ ]:
for i in range(len(gdf_building_points)):
    building_ufi=gdf_building_points.iloc[i]['UFI']
    try:
        las_file_path=glob.glob(las_files_folder+'*'+'_UFI_'+str(building_ufi)+'.las')[0]
        output_tiff=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_DTM_filtered.tif')) # output file name

        # Get CRS from source file first
        pipeline_info = {
            "pipeline": [{
                "type": "readers.las",
                "filename": las_file_path
            }]
        }
        info_pipeline = pdal.Pipeline(json.dumps(pipeline_info))
        info_pipeline.execute()
        metadata = info_pipeline.metadata
        crs = metadata['metadata']['readers.las']['srs']['horizontal']

        stats=process_extract_ground_elevations(las_file_path, resolution,crs=crs, output_tiff=output_tiff)

        for attr in list(stats.keys()):
            if attr not in gdf_updated.columns:
                gdf_updated[attr]=np.nan
        for attr in list(stats.keys()):
            gdf_updated.at[i,attr]=stats[attr]
    except Exception as e:
        print(e)

### Save stats to building points

In [ ]:
gdf_updated.to_file(out_building_points)

### Plot distributions

In [ ]:
# gdf_updated = gpd.read_file(out_building_points)
column_to_plot = 'lidar_elev_std'
# Set up the figure
plt.figure(figsize=(20, 6))
# Plot histogram
plt.subplot(2, 2, 1)
gdf_updated[column_to_plot].hist(bins=200)
plt.title('Std of filtered ground elevations (m)')

In [ ]:
# gdf_updated = gpd.read_file(out_building_points)
column_to_plot = 'lidar_elev_std'
# Set up the figure
plt.figure(figsize=(20, 6))
# Plot histogram
plt.subplot(2, 2, 1)
(gdf_updated['lidar_elev_max'] - gdf_updated['lidar_elev_min']).hist(bins=200)
plt.title('Range of filtered ground elevations (m)')